In [1]:
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train=pd.read_csv("C:/Users/Shafiq/Desktop/arsh LU/final_train.csv")
test=pd.read_csv("C:/Users/Shafiq/Desktop/arsh LU/final_test.csv")

In [3]:
train

,source_node,destination_node,Jaccard_followees,Jaccard_followers,cosine_followees,cosine_followers,Avg_sim_followees,Avg_sim_followers,Adar Index,Is_Followback,...,katz_d,hub_source,auth_source,hub_dest,auth_dest,wt_f1,wt_f2,wt_f3,wt_f4,link_indicator
0,128335,990238,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001024,1,1,1,1,0.544331,0.055556,0.952579,0.680414,1
1,64311,874986,0.000000,0.333333,0,0,0.00,0.17,0.000000,1,...,0.001008,1,1,1,1,0.813053,0.136083,1.390403,1.048755,1
2,108983,1416606,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001008,1,1,1,1,0.910684,0.192450,1.488034,1.244017,1
3,17882,1789054,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001003,1,1,1,1,1.207107,0.353553,1.914214,1.707107,1
4,109045,85458,0.052632,0.000000,0,0,0.03,0.00,2.183295,1,...,0.001024,1,1,1,1,0.597231,0.077152,1.005479,0.786213,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677715,633464,211229,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001034,1,1,1,1,0.853553,0.176777,1.207107,1.353553,0
1677716,498101,196926,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001003,1,1,1,1,1.414214,0.500000,2.121320,2.121320,0
1677717,183408,179009,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001018,1,1,1,1,0.855462,0.182574,1.302675,1.263710,0
1677718,233512,554483,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001013,1,1,1,1,1.207107,0.353553,1.707107,1.914214,0


# Adding few more features to train and test dataset

1) Weakly Connected Components
2) Katz Centerality
3) Hitz Score(hub and authority score)
4) Some Features on the basis of Weight Indexes


In [4]:
import networkx as nx

# For train

In [5]:
train_graph=nx.from_pandas_edgelist(train,source="source_node",target="destination_node",create_using=nx.DiGraph)

print(nx.info(train_graph))

C:\Users\Shafiq\AppData\Local\Temp\ipykernel_12888\2897728654.py:3: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(train_graph))


DiGraph with 986384 nodes and 1677720 edges


# Weakly Connected Components

In [6]:
wcc=list(nx.weakly_connected_components(train_graph))
def is_same_wcc(n1,n2):
    index = []
    if train_graph.has_edge(n2,n1):
        return 1
    if train_graph.has_edge(n1,n2):
            for i in wcc:
                if n1 in i:
                    index= i
                    break
            if (n2 in index):
                train_graph.remove_edge(a,b)
                if compute_shortest_path_length(n1,n2)==-1:
                    train_graph.add_edge(n1,n2)
                    return 0
                else:
                    train_graph.add_edge(n1,n2)
                    return 1
            else:
                return 0
    else:
            for i in wcc:
                if n1 in i:
                    index= i
                    break
            if(n2 in index):
                return 1
            else:
                return 0

In [7]:
len(wcc)

5387

In [8]:
wcc[3]

{1635, 930800, 1582681, 1718813, 1809093}

In [9]:
train['same_comp'] = train.apply(lambda row: is_same_wcc(row["destination_node"],row['source_node']),axis=1)

In [10]:
train

,source_node,destination_node,Jaccard_followees,Jaccard_followers,cosine_followees,cosine_followers,Avg_sim_followees,Avg_sim_followers,Adar Index,Is_Followback,...,katz_d,hub_source,auth_source,hub_dest,auth_dest,wt_f1,wt_f2,wt_f3,wt_f4,link_indicator
0,128335,990238,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001024,1,1,1,1,0.544331,0.055556,0.952579,0.680414,1
1,64311,874986,0.000000,0.333333,0,0,0.00,0.17,0.000000,1,...,0.001008,1,1,1,1,0.813053,0.136083,1.390403,1.048755,1
2,108983,1416606,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001008,1,1,1,1,0.910684,0.192450,1.488034,1.244017,1
3,17882,1789054,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001003,1,1,1,1,1.207107,0.353553,1.914214,1.707107,1
4,109045,85458,0.052632,0.000000,0,0,0.03,0.00,2.183295,1,...,0.001024,1,1,1,1,0.597231,0.077152,1.005479,0.786213,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677715,633464,211229,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001034,1,1,1,1,0.853553,0.176777,1.207107,1.353553,0
1677716,498101,196926,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001003,1,1,1,1,1.414214,0.500000,2.121320,2.121320,0
1677717,183408,179009,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001018,1,1,1,1,0.855462,0.182574,1.302675,1.263710,0
1677718,233512,554483,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001013,1,1,1,1,1.207107,0.353553,1.707107,1.914214,0


# Katz Centerality

In [11]:
katz = nx.katz.katz_centrality(train_graph,alpha=0.005,beta=1)
print('min',katz[min(katz, key=katz.get)])
print('max',katz[max(katz, key=katz.get)])
print('mean',float(sum(katz.values())) / len(katz))

min 0.000998260208090097
max 0.0012447514809189584
mean 0.0010068492068495882


In [12]:
mean_katz = float(sum(katz.values())) / len(katz)

train['katz_s'] = train.source_node.apply(lambda x: katz.get(x,mean_katz))
train['katz_d'] = train.destination_node.apply(lambda x: katz.get(x,mean_katz))

In [13]:
train

,source_node,destination_node,Jaccard_followees,Jaccard_followers,cosine_followees,cosine_followers,Avg_sim_followees,Avg_sim_followers,Adar Index,Is_Followback,...,katz_d,hub_source,auth_source,hub_dest,auth_dest,wt_f1,wt_f2,wt_f3,wt_f4,link_indicator
0,128335,990238,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001024,1,1,1,1,0.544331,0.055556,0.952579,0.680414,1
1,64311,874986,0.000000,0.333333,0,0,0.00,0.17,0.000000,1,...,0.001008,1,1,1,1,0.813053,0.136083,1.390403,1.048755,1
2,108983,1416606,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001008,1,1,1,1,0.910684,0.192450,1.488034,1.244017,1
3,17882,1789054,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001003,1,1,1,1,1.207107,0.353553,1.914214,1.707107,1
4,109045,85458,0.052632,0.000000,0,0,0.03,0.00,2.183295,1,...,0.001024,1,1,1,1,0.597231,0.077152,1.005479,0.786213,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677715,633464,211229,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001034,1,1,1,1,0.853553,0.176777,1.207107,1.353553,0
1677716,498101,196926,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001003,1,1,1,1,1.414214,0.500000,2.121320,2.121320,0
1677717,183408,179009,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001018,1,1,1,1,0.855462,0.182574,1.302675,1.263710,0
1677718,233512,554483,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001013,1,1,1,1,1.207107,0.353553,1.707107,1.914214,0


# Hitz Score

In [14]:
hub={}
auth={}

n_auth=1

n_hub=1

for index,row in train.iterrows():
    
    source=int(row["source_node"])
    dest=int(row["destination_node"])
    
    hub[source]=1
    
    auth[source]=1
    
    hub[dest]=1
    
    auth[dest]=1


     
for index,row in train.iterrows():
    
    source=int(row["source_node"])
    dest=int(row["destination_node"])
    
        
    for p in set(train_graph.predecessors(source)):
        
        n_auth=0
        n_auth= n_auth+hub[p]
    
    auth[source]=n_auth
    
    for p in set(train_graph.predecessors(dest)):
        
        n_auth=0
        n_auth= n_auth+hub[p]
    
    auth[dest]=n_auth
    
    for s in set(train_graph.successors(source)):
        
        n_hub=0
        
        n_hub=n_hub+auth[s]
    
    
    hub[source]=n_hub

    for s in set(train_graph.successors(dest)):
        
        n_hub=0
        
        n_hub=n_hub+auth[s]
    
    
    hub[dest]=n_hub
                



In [15]:
hub_source=[]
auth_source=[]
hub_dest=[]
auth_dest=[]

for index,row in train.iterrows():
    
    source=int(row["source_node"])
    dest=int(row["destination_node"])
    
    hub_source.append(hub[source])
    auth_source.append(auth[source])
    hub_dest.append(hub[dest])
    auth_dest.append(auth[dest])

In [16]:
train["hub_source"]=hub_source
train["auth_source"]=auth_source
train["hub_dest"]=hub_dest
train["auth_dest"]= auth_dest


In [17]:
train

,source_node,destination_node,Jaccard_followees,Jaccard_followers,cosine_followees,cosine_followers,Avg_sim_followees,Avg_sim_followers,Adar Index,Is_Followback,...,katz_d,hub_source,auth_source,hub_dest,auth_dest,wt_f1,wt_f2,wt_f3,wt_f4,link_indicator
0,128335,990238,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001024,1,1,1,1,0.544331,0.055556,0.952579,0.680414,1
1,64311,874986,0.000000,0.333333,0,0,0.00,0.17,0.000000,1,...,0.001008,1,1,1,1,0.813053,0.136083,1.390403,1.048755,1
2,108983,1416606,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001008,1,1,1,1,0.910684,0.192450,1.488034,1.244017,1
3,17882,1789054,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001003,1,1,1,1,1.207107,0.353553,1.914214,1.707107,1
4,109045,85458,0.052632,0.000000,0,0,0.03,0.00,2.183295,1,...,0.001024,1,1,1,1,0.597231,0.077152,1.005479,0.786213,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677715,633464,211229,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001034,1,1,1,1,0.853553,0.176777,1.207107,1.353553,0
1677716,498101,196926,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001003,1,1,1,1,1.414214,0.500000,2.121320,2.121320,0
1677717,183408,179009,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001018,1,1,1,1,0.855462,0.182574,1.302675,1.263710,0
1677718,233512,554483,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001013,1,1,1,1,1.207107,0.353553,1.707107,1.914214,0


# Features on basis of Weight Index

In [18]:
train['wt_f1'] = train.weight_in + train.weight_out
train['wt_f2'] = train.weight_in * train.weight_out
train['wt_f3'] = (2*train.weight_in +train.weight_out)
train['wt_f4'] = (train.weight_in + 2*train.weight_out)

In [19]:
link_indicator=train["link_indicator"]

train.drop("link_indicator",axis=1,inplace=True)

train["link_indicator"]=link_indicator

In [20]:
train

,source_node,destination_node,Jaccard_followees,Jaccard_followers,cosine_followees,cosine_followers,Avg_sim_followees,Avg_sim_followers,Adar Index,Is_Followback,...,katz_d,hub_source,auth_source,hub_dest,auth_dest,wt_f1,wt_f2,wt_f3,wt_f4,link_indicator
0,128335,990238,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001024,1,1,1,1,0.544331,0.055556,0.952579,0.680414,1
1,64311,874986,0.000000,0.333333,0,0,0.00,0.17,0.000000,1,...,0.001008,1,1,1,1,0.813053,0.136083,1.390403,1.048755,1
2,108983,1416606,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001008,1,1,1,1,0.910684,0.192450,1.488034,1.244017,1
3,17882,1789054,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001003,1,1,1,1,1.207107,0.353553,1.914214,1.707107,1
4,109045,85458,0.052632,0.000000,0,0,0.03,0.00,2.183295,1,...,0.001024,1,1,1,1,0.597231,0.077152,1.005479,0.786213,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677715,633464,211229,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001034,1,1,1,1,0.853553,0.176777,1.207107,1.353553,0
1677716,498101,196926,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001003,1,1,1,1,1.414214,0.500000,2.121320,2.121320,0
1677717,183408,179009,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001018,1,1,1,1,0.855462,0.182574,1.302675,1.263710,0
1677718,233512,554483,0.000000,0.000000,0,0,0.00,0.00,0.000000,1,...,0.001013,1,1,1,1,1.207107,0.353553,1.707107,1.914214,0


# For test

In [21]:
test_graph=nx.from_pandas_edgelist(test,source="source_node",target="destination_node",create_using=nx.DiGraph)

print(nx.info(test_graph))

C:\Users\Shafiq\AppData\Local\Temp\ipykernel_12888\575802934.py:3: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(test_graph))


DiGraph with 504490 nodes and 419430 edges


# Weekly connected Components

In [22]:
wcc=list(nx.weakly_connected_components(test_graph))
def is_same_wcc(n1,n2):
    index = []
    if test_graph.has_edge(n2,n1):
        return 1
    if test_graph.has_edge(n1,n2):
            for i in wcc:
                if n1 in i:
                    index= i
                    break
            if (n2 in index):
                test_graph.remove_edge(a,b)
                if compute_shortest_path_length(n1,n2)==-1:
                    test_graph.add_edge(n1,n2)
                    return 0
                else:
                    test_graph.add_edge(n1,n2)
                    return 1
            else:
                return 0
    else:
            for i in wcc:
                if n1 in i:
                    index= i
                    break
            if(n2 in index):
                return 1
            else:
                return 0
            
len(wcc)

101425

In [23]:
test['same_comp'] = test.apply(lambda row: is_same_wcc(row["destination_node"],row['source_node']),axis=1)

In [24]:
test

,source_node,destination_node,Jaccard_followees,Jaccard_followers,cosine_followees,cosine_followers,Avg_sim_followees,Avg_sim_followers,Adar Index,Is_Followback,...,katz_d,hub_source,auth_source,hub_dest,auth_dest,wt_f1,wt_f2,wt_f3,wt_f4,link_indicator
0,204446,139592,0.0,0.0,0,0,0.0,0.0,0.0,0,...,0.001409,1,1,1,1,1.577350,0.577350,2.577350,2.154701,1
1,70119,256428,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.414214,0.500000,2.121320,2.121320,1
2,171448,1548414,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.154320,0.316228,1.861427,1.601534,1
3,108453,163467,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.023335,0.223607,1.730441,1.339562,1
4,53271,1548358,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.207107,0.353553,1.914214,1.707107,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419425,425062,515552,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001416,1,1,1,1,1.284457,0.408248,1.861807,1.991564,0
419426,380571,381467,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.284457,0.408248,1.991564,1.861807,0
419427,585852,270912,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001416,1,1,1,1,1.284457,0.408248,1.861807,1.991564,0
419428,472340,224297,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001416,1,1,1,1,1.284457,0.408248,1.861807,1.991564,0


# Katz Centerality

In [25]:
katz = nx.katz.katz_centrality(test_graph,alpha=0.005,beta=1)
print('min',katz[min(katz, key=katz.get)])
print('max',katz[max(katz, key=katz.get)])
print('mean',float(sum(katz.values())) / len(katz))

min 0.0014020514473704066
max 0.0015166044835699275
mean 0.0014078963643765355


In [26]:
mean_katz = float(sum(katz.values())) / len(katz)

test['katz_s'] = test.source_node.apply(lambda x: katz.get(x,mean_katz))
test['katz_d'] = test.destination_node.apply(lambda x: katz.get(x,mean_katz))

test

,source_node,destination_node,Jaccard_followees,Jaccard_followers,cosine_followees,cosine_followers,Avg_sim_followees,Avg_sim_followers,Adar Index,Is_Followback,...,katz_d,hub_source,auth_source,hub_dest,auth_dest,wt_f1,wt_f2,wt_f3,wt_f4,link_indicator
0,204446,139592,0.0,0.0,0,0,0.0,0.0,0.0,0,...,0.001409,1,1,1,1,1.577350,0.577350,2.577350,2.154701,1
1,70119,256428,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.414214,0.500000,2.121320,2.121320,1
2,171448,1548414,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.154320,0.316228,1.861427,1.601534,1
3,108453,163467,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.023335,0.223607,1.730441,1.339562,1
4,53271,1548358,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.207107,0.353553,1.914214,1.707107,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419425,425062,515552,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001416,1,1,1,1,1.284457,0.408248,1.861807,1.991564,0
419426,380571,381467,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.284457,0.408248,1.991564,1.861807,0
419427,585852,270912,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001416,1,1,1,1,1.284457,0.408248,1.861807,1.991564,0
419428,472340,224297,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001416,1,1,1,1,1.284457,0.408248,1.861807,1.991564,0


# Hitz Score

In [27]:
hub={}
auth={}

n_auth=1

n_hub=1

for index,row in test.iterrows():
    
    source=int(row["source_node"])
    dest=int(row["destination_node"])
    
    hub[source]=1
    
    auth[source]=1
    
    hub[dest]=1
    
    auth[dest]=1


     
for index,row in test.iterrows():
    
    source=int(row["source_node"])
    dest=int(row["destination_node"])
    
        
    for p in set(test_graph.predecessors(source)):
        
        n_auth=0
        n_auth= n_auth+hub[p]
    
    auth[source]=n_auth
    
    for p in set(test_graph.predecessors(dest)):
        
        n_auth=0
        n_auth= n_auth+hub[p]
    
    auth[dest]=n_auth
    
    for s in set(test_graph.successors(source)):
        
        n_hub=0
        
        n_hub=n_hub+auth[s]
    
    
    hub[source]=n_hub

    for s in set(test_graph.successors(dest)):
        
        n_hub=0
        
        n_hub=n_hub+auth[s]
    
    
    hub[dest]=n_hub
                


In [28]:
hub_source=[]
auth_source=[]
hub_dest=[]
auth_dest=[]

for index,row in test.iterrows():
    
    source=int(row["source_node"])
    dest=int(row["destination_node"])
    
    hub_source.append(hub[source])
    auth_source.append(auth[source])
    hub_dest.append(hub[dest])
    auth_dest.append(auth[dest])

In [29]:
test["hub_source"]=hub_source
test["auth_source"]=auth_source
test["hub_dest"]=hub_dest
test["auth_dest"]= auth_dest

test


,source_node,destination_node,Jaccard_followees,Jaccard_followers,cosine_followees,cosine_followers,Avg_sim_followees,Avg_sim_followers,Adar Index,Is_Followback,...,katz_d,hub_source,auth_source,hub_dest,auth_dest,wt_f1,wt_f2,wt_f3,wt_f4,link_indicator
0,204446,139592,0.0,0.0,0,0,0.0,0.0,0.0,0,...,0.001409,1,1,1,1,1.577350,0.577350,2.577350,2.154701,1
1,70119,256428,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.414214,0.500000,2.121320,2.121320,1
2,171448,1548414,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.154320,0.316228,1.861427,1.601534,1
3,108453,163467,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.023335,0.223607,1.730441,1.339562,1
4,53271,1548358,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.207107,0.353553,1.914214,1.707107,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419425,425062,515552,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001416,1,1,1,1,1.284457,0.408248,1.861807,1.991564,0
419426,380571,381467,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.284457,0.408248,1.991564,1.861807,0
419427,585852,270912,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001416,1,1,1,1,1.284457,0.408248,1.861807,1.991564,0
419428,472340,224297,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001416,1,1,1,1,1.284457,0.408248,1.861807,1.991564,0


# Some Features on basis of Weight Indexes

In [30]:
test['wt_f1'] = test.weight_in + test.weight_out
test['wt_f2'] = test.weight_in * test.weight_out
test['wt_f3'] = (2*test.weight_in +test.weight_out)
test['wt_f4'] = (test.weight_in + 2*test.weight_out)

test

,source_node,destination_node,Jaccard_followees,Jaccard_followers,cosine_followees,cosine_followers,Avg_sim_followees,Avg_sim_followers,Adar Index,Is_Followback,...,katz_d,hub_source,auth_source,hub_dest,auth_dest,wt_f1,wt_f2,wt_f3,wt_f4,link_indicator
0,204446,139592,0.0,0.0,0,0,0.0,0.0,0.0,0,...,0.001409,1,1,1,1,1.577350,0.577350,2.577350,2.154701,1
1,70119,256428,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.414214,0.500000,2.121320,2.121320,1
2,171448,1548414,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.154320,0.316228,1.861427,1.601534,1
3,108453,163467,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.023335,0.223607,1.730441,1.339562,1
4,53271,1548358,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.207107,0.353553,1.914214,1.707107,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419425,425062,515552,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001416,1,1,1,1,1.284457,0.408248,1.861807,1.991564,0
419426,380571,381467,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.284457,0.408248,1.991564,1.861807,0
419427,585852,270912,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001416,1,1,1,1,1.284457,0.408248,1.861807,1.991564,0
419428,472340,224297,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001416,1,1,1,1,1.284457,0.408248,1.861807,1.991564,0


In [31]:
link_indicator=test["link_indicator"]

test.drop("link_indicator",axis=1,inplace=True)

test["link_indicator"]=link_indicator

test

,source_node,destination_node,Jaccard_followees,Jaccard_followers,cosine_followees,cosine_followers,Avg_sim_followees,Avg_sim_followers,Adar Index,Is_Followback,...,katz_d,hub_source,auth_source,hub_dest,auth_dest,wt_f1,wt_f2,wt_f3,wt_f4,link_indicator
0,204446,139592,0.0,0.0,0,0,0.0,0.0,0.0,0,...,0.001409,1,1,1,1,1.577350,0.577350,2.577350,2.154701,1
1,70119,256428,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.414214,0.500000,2.121320,2.121320,1
2,171448,1548414,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.154320,0.316228,1.861427,1.601534,1
3,108453,163467,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.023335,0.223607,1.730441,1.339562,1
4,53271,1548358,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.207107,0.353553,1.914214,1.707107,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419425,425062,515552,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001416,1,1,1,1,1.284457,0.408248,1.861807,1.991564,0
419426,380571,381467,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001409,1,1,1,1,1.284457,0.408248,1.991564,1.861807,0
419427,585852,270912,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001416,1,1,1,1,1.284457,0.408248,1.861807,1.991564,0
419428,472340,224297,0.0,0.0,0,0,0.0,0.0,0.0,1,...,0.001416,1,1,1,1,1.284457,0.408248,1.861807,1.991564,0


# Saving the dataset as csv

In [32]:
train.to_csv("C:/Users/Shafiq/Desktop/arsh LU/final_train.csv",index=False)

test.to_csv("C:/Users/Shafiq/Desktop/arsh LU/final_test.csv",index=False)